# Phân loại cảm xúc IMDB sử dụng BiLSTM + Attention

Trong notebook này, chúng ta sẽ xây dựng mô hình BiLSTM kết hợp với lớp Attention để phân loại cảm xúc (tích cực/tiêu cực) trên bộ dữ liệu IMDb movie reviews.

## Mục tiêu:
- Xây dựng mô hình BiLSTM + Attention từ đầu
- Huấn luyện trên dataset IMDb 
- Đánh giá hiệu quả mô hình
- So sánh với các phương pháp khác

## Các bước thực hiện:
1. Import thư viện và thiết lập môi trường
2. Load và tiền xử lý dữ liệu IMDb
3. Xây dựng lớp Attention tùy chỉnh
4. Tạo mô hình BiLSTM + Attention
5. Huấn luyện mô hình
6. Đánh giá và trực quan hóa kết quả
7. Test với dữ liệu mới

### 1. Import thư viện và thiết lập môi trường

Import các thư viện cần thiết cho việc xây dựng mô hình BiLSTM + Attention:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Thiết lập seed cho reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.13.0
GPU available: []


### 2. Tạo lớp Attention tùy chỉnh

Xây dựng lớp Attention để tăng khả năng tập trung vào những phần quan trọng trong chuỗi văn bản:

In [2]:
class AttentionLayer(layers.Layer):
    """
    Custom Attention Layer cho BiLSTM
    Cho phép mô hình tập trung vào các phần quan trọng trong chuỗi đầu vào
    """
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Ma trận trọng số attention
        self.W = self.add_weight(name="attention_weight", 
                                shape=(input_shape[-1], 1),
                                initializer='random_normal',
                                trainable=True)
        # Bias cho attention
        self.b = self.add_weight(name="attention_bias", 
                                shape=(input_shape[1], 1),
                                initializer='zeros',
                                trainable=True)        
        super(AttentionLayer, self).build(input_shape)
        
    def call(self, x):
        # x shape: (batch_size, time_steps, features)
        # Tính attention scores
        e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        # Chuẩn hóa attention weights bằng softmax
        a = tf.nn.softmax(e, axis=1)
        
        # Áp dụng attention weights lên input
        output = x * a
        
        # Tổng hợp thông tin từ tất cả time steps
        return tf.reduce_sum(output, axis=1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

print("Lớp AttentionLayer đã được định nghĩa thành công!")

Lớp AttentionLayer đã được định nghĩa thành công!


### 3. Load và tiền xử lý dữ liệu IMDb

Tải bộ dữ liệu IMDb movie reviews và thực hiện tiền xử lý:

In [3]:
def load_and_preprocess_data(max_features=10000, maxlen=500):
    """
    Load và tiền xử lý dữ liệu IMDb
    """
    print("Đang tải dữ liệu IMDb...")
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
    
    print(f"Số lượng reviews training: {len(x_train)}")
    print(f"Số lượng reviews testing: {len(x_test)}")
    print(f"Số lượng từ tối đa: {max_features}")
    print(f"Độ dài sequence tối đa: {maxlen}")
    
    # Kiểm tra phân bố nhãn
    print(f"\nPhân bố nhãn training:")
    print(f"- Negative (0): {sum(y_train == 0)} reviews")
    print(f"- Positive (1): {sum(y_train == 1)} reviews")
    
    # Padding sequences để có cùng độ dài
    print("\nĐang padding sequences...")
    x_train = pad_sequences(x_train, maxlen=maxlen)
    x_test = pad_sequences(x_test, maxlen=maxlen)
    
    print(f"Shape sau padding:")
    print(f"x_train: {x_train.shape}")
    print(f"x_test: {x_test.shape}")
    
    return (x_train, y_train), (x_test, y_test)

# Thiết lập tham số
MAX_FEATURES = 10000  # Số từ vựng tối đa
MAXLEN = 500         # Độ dài sequence tối đa

# Load dữ liệu
(x_train, y_train), (x_test, y_test) = load_and_preprocess_data(MAX_FEATURES, MAXLEN)

Đang tải dữ liệu IMDb...
17464789/17464789 [==============================] - 12s 1us/step
Số lượng reviews training: 25000
Số lượng reviews testing: 25000
Số lượng từ tối đa: 10000
Độ dài sequence tối đa: 500

Phân bố nhãn training:
- Negative (0): 12500 reviews
- Positive (1): 12500 reviews

Đang padding sequences...
Shape sau padding:
x_train: (25000, 500)
x_test: (25000, 500)


### 4. Xây dựng mô hình BiLSTM + Attention

Tạo mô hình sử dụng lớp BiLSTM kết hợp với Attention mechanism:

In [4]:
def create_bilstm_attention_model(max_features, maxlen, embedding_dim=128, lstm_units=64):
    """
    Tạo model BiLSTM với Attention
    """
    model = models.Sequential()
    
    # Lớp Embedding: chuyển từ số thành vector đặc trưng
    model.add(layers.Embedding(max_features, embedding_dim, input_length=maxlen))
    model.add(layers.Dropout(0.2))
    
    # Hai lớp BiLSTM để học đặc trưng từ cả hai hướng
    model.add(layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)
    ))
    model.add(layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)
    ))
    
    # Lớp Attention để tập trung vào phần quan trọng
    model.add(AttentionLayer())
    model.add(layers.Dropout(0.5))
    
    # Các lớp Dense để phân loại
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dropout(0.3))
    
    # Lớp output: phân loại nhị phân (positive/negative)
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

# Tham số mô hình
EMBEDDING_DIM = 128  # Dimension của embedding
LSTM_UNITS = 64      # Số units trong LSTM

# Tạo mô hình
print("Đang xây dựng model...")
model = create_bilstm_attention_model(MAX_FEATURES, MAXLEN, EMBEDDING_DIM, LSTM_UNITS)

# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Hiển thị kiến trúc mô hình
print("\nKiến trúc mô hình:")
model.summary()

Đang xây dựng model...



Kiến trúc mô hình:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 dropout (Dropout)           (None, 500, 128)          0         
                                                                 
 bidirectional (Bidirection  (None, 500, 128)          98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 500, 128)          98816     
 onal)                                                           
                                                                 
 attention_layer (Attention  (None, 128)               628       
 Layer)                                                          
                                    

### 5. Huấn luyện mô hình

Thiết lập callbacks và bắt đầu quá trình huấn luyện:

In [ ]:
# Tham số huấn luyện
BATCH_SIZE = 32      # Batch size
EPOCHS = 10          # Số epochs

# Thiết lập callbacks để tối ưu hóa quá trình huấn luyện
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)
]

# Bắt đầu huấn luyện
print("Bắt đầu huấn luyện mô hình...")
print(f"Batch size: {BATCH_SIZE}")
print(f"Max epochs: {EPOCHS}")
print(f"Callbacks: EarlyStopping, ReduceLROnPlateau")

history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(x_test, y_test),
    callbacks=callbacks,
    verbose=1
)

print("\nHuấn luyện hoàn tất!")

Bắt đầu huấn luyện mô hình...
Batch size: 32
Max epochs: 10
Callbacks: EarlyStopping, ReduceLROnPlateau
Epoch 1/10
221/782 [=======>......................] - ETA: 7:49 - loss: 0.6513 - accuracy: 0.5703

### 6. Trực quan hóa quá trình huấn luyện

Vẽ biểu đồ để quan sát quá trình huấn luyện:

In [ ]:
def plot_training_history(history):
    """
    Vẽ biểu đồ quá trình training
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training & validation accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    ax1.set_title('Độ chính xác mô hình qua các epoch')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot training & validation loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    ax2.set_title('Loss mô hình qua các epoch')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Vẽ biểu đồ
plot_training_history(history)

### 7. Đánh giá mô hình

Đánh giá hiệu quả mô hình trên tập test và hiển thị các metrics chi tiết:

In [ ]:
def evaluate_model(model, x_test, y_test):
    """
    Đánh giá model và hiển thị kết quả chi tiết
    """
    # Dự đoán trên tập test
    print("Đang thực hiện dự đoán trên tập test...")
    y_pred_prob = model.predict(x_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Classification report
    print("\n" + "="*60)
    print("BÁO CÁO PHÂN LOẠI CHI TIẾT")
    print("="*60)
    print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Negative', 'Positive'], 
                yticklabels=['Negative', 'Positive'])
    plt.title('Ma trận nhầm lẫn (Confusion Matrix)')
    plt.xlabel('Dự đoán')
    plt.ylabel('Thực tế')
    plt.show()
    
    # Test accuracy
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"\n{'='*60}")
    print(f"KẾT QUẢ CUỐI CÙNG")
    print(f"{'='*60}")
    print(f"Độ chính xác trên tập test: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
    print(f"Loss trên tập test: {test_loss:.4f}")
    
    return y_pred, y_pred_prob

# Đánh giá mô hình
y_pred, y_pred_prob = evaluate_model(model, x_test, y_test)

### 8. Test với dữ liệu mới

Thử nghiệm mô hình với một số reviews mẫu để xem khả năng dự đoán:

In [ ]:
def predict_sentiment(model, text, word_index, maxlen=500):
    """
    Dự đoán cảm xúc cho một text mới
    """
    # Chuyển text thành sequence
    sequence = []
    for word in text.lower().split():
        if word in word_index and word_index[word] < 10000:
            sequence.append(word_index[word])
    
    # Nếu sequence rỗng, return unknown
    if not sequence:
        print(f"Text: {text}")
        print("Không thể xử lý text này (không có từ nào trong từ điển)")
        print("-" * 50)
        return
    
    # Padding
    sequence = pad_sequences([sequence], maxlen=maxlen)
    
    # Predict
    prediction = model.predict(sequence, verbose=0)[0][0]
    
    sentiment = "Tích cực (Positive)" if prediction > 0.5 else "Tiêu cực (Negative)"
    confidence = prediction if prediction > 0.5 else 1 - prediction
    
    print(f"Text: {text}")
    print(f"Cảm xúc: {sentiment}")
    print(f"Độ tin cậy: {confidence:.4f} ({confidence*100:.2f}%)")
    print(f"Điểm số raw: {prediction:.4f}")
    print("-" * 50)

# Load word index để chuyển đổi text
word_index = imdb.get_word_index()

print("KIỂM THỬ MÔ HÌNH VỚI CÁC REVIEWS MẪU")
print("="*60)

# Test với các examples khác nhau
test_texts = [
    "This movie is absolutely fantastic! Great acting and amazing storyline.",
    "Terrible film. Boring plot and bad acting. Complete waste of time.",
    "The movie was okay, nothing special but not bad either.",
    "One of the best movies I have ever seen! Highly recommended!",
    "Awful movie. Poor direction and terrible script. Very disappointed.",
    "Amazing cinematography and outstanding performances from all actors.",
    "I fell asleep halfway through. Very boring and predictable plot.",
    "Perfect movie for the weekend. Really enjoyed watching it.",
    "Not my cup of tea but can understand why others might like it.",
    "Masterpiece! Every scene was perfectly crafted and emotionally engaging."
]

for text in test_texts:
    predict_sentiment(model, text, word_index, MAXLEN)

### 9. Lưu mô hình

Lưu lại mô hình đã huấn luyện để sử dụng sau này:

In [ ]:
# Lưu mô hình
model.save('bilstm_attention_imdb_model.h5')
print("✅ Mô hình đã được lưu thành 'bilstm_attention_imdb_model.h5'")

# Lưu thông tin về quá trình huấn luyện
import json

training_info = {
    'max_features': MAX_FEATURES,
    'maxlen': MAXLEN,
    'embedding_dim': EMBEDDING_DIM,
    'lstm_units': LSTM_UNITS,
    'batch_size': BATCH_SIZE,
    'epochs_trained': len(history.history['loss']),
    'final_train_accuracy': float(history.history['accuracy'][-1]),
    'final_val_accuracy': float(history.history['val_accuracy'][-1]),
    'final_train_loss': float(history.history['loss'][-1]),
    'final_val_loss': float(history.history['val_loss'][-1])
}

with open('training_info.json', 'w') as f:
    json.dump(training_info, f, indent=2)

print("✅ Thông tin huấn luyện đã được lưu vào 'training_info.json'")
print("\nThông tin mô hình:")
for key, value in training_info.items():
    if 'accuracy' in key or 'loss' in key:
        print(f"- {key}: {value:.4f}")
    else:
        print(f"- {key}: {value}")

### 10. Tóm tắt và so sánh

#### Ưu điểm của mô hình BiLSTM + Attention:
- **Hiểu ngữ cảnh hai chiều**: BiLSTM xử lý thông tin từ cả hai hướng (trước và sau)
- **Tập trung vào phần quan trọng**: Lớp Attention giúp mô hình chú ý đến những từ quan trọng nhất
- **Hiệu quả với dữ liệu tuần tự**: Phù hợp với đặc tính của văn bản là dữ liệu tuần tự
- **Tốc độ huấn luyện hợp lý**: Nhanh hơn so với các mô hình Transformer lớn

#### Nhược điểm:
- **Độ chính xác thấp hơn BERT**: Có thể không đạt được độ chính xác cao như các mô hình pretrained lớn
- **Cần nhiều dữ liệu**: Hiệu quả tốt nhất khi có lượng dữ liệu huấn luyện lớn
- **Khó xử lý mối quan hệ xa**: LSTM có thể gặp khó khăn với các phụ thuộc xa trong văn bản

#### So sánh với BERT:
- **BERT**: Độ chính xác cao hơn nhưng cần nhiều tài nguyên tính toán và thời gian huấn luyện
- **BiLSTM + Attention**: Cân bằng giữa hiệu quả và tốc độ, phù hợp khi tài nguyên hạn chế

Mô hình BiLSTM + Attention là một lựa chọn tốt khi cần cân bằng giữa độ chính xác và hiệu quả tính toán!

### 11. Tạo thư mục và lưu mô hình

Tạo thư mục `bilstm` và lưu mô hình cùng với các thông tin liên quan:

In [ ]:
import os
import json
import time
import psutil
from datetime import datetime

# Tạo thư mục bilstm nếu chưa có
os.makedirs('bilstm', exist_ok=True)

# Lưu mô hình vào thư mục bilstm
model.save('bilstm/bilstm_attention_model.h5')
print("✅ Mô hình đã được lưu vào 'bilstm/bilstm_attention_model.h5'")

# Lưu tokenizer info (word_index) để sử dụng sau này
import pickle
with open('bilstm/word_index.pkl', 'wb') as f:
    pickle.dump(word_index, f)
print("✅ Word index đã được lưu vào 'bilstm/word_index.pkl'")

### 12. Tính toán các thông số đánh giá chi tiết

#### 12.1 Độ chính xác (Accuracy Metrics)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

# Tính toán các metrics chính xác
print("Đang tính toán các thông số đánh giá...")

# 1. ACCURACY METRICS
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_pred_prob)

# Confusion Matrix chi tiết
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

# Tính toán thêm các metrics
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)  # = recall
false_positive_rate = fp / (fp + tn)
false_negative_rate = fn / (fn + tp)

print("1. ACCURACY METRICS:")
print(f"   - Accuracy: {accuracy:.4f}")
print(f"   - Precision: {precision:.4f}")
print(f"   - Recall (Sensitivity): {recall:.4f}")
print(f"   - Specificity: {specificity:.4f}")
print(f"   - F1-Score: {f1:.4f}")
print(f"   - AUC-ROC: {auc_roc:.4f}")
print(f"   - False Positive Rate: {false_positive_rate:.4f}")
print(f"   - False Negative Rate: {false_negative_rate:.4f}")

accuracy_metrics = {
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'specificity': float(specificity),
    'f1_score': float(f1),
    'auc_roc': float(auc_roc),
    'false_positive_rate': float(false_positive_rate),
    'false_negative_rate': float(false_negative_rate),
    'true_positives': int(tp),
    'true_negatives': int(tn),
    'false_positives': int(fp),
    'false_negatives': int(fn)
}

#### 12.2 Hiệu suất tính toán (Performance Metrics)

In [ ]:
# 2. PERFORMANCE METRICS

# Tính thời gian training (ước lượng từ history)
epochs_trained = len(history.history['loss'])
# Ước lượng thời gian training (có thể không chính xác 100%)
estimated_training_time = epochs_trained * 60  # Ước lượng 60s/epoch

# Đo thời gian inference
print("Đang đo thời gian inference...")
inference_times = []
batch_size_test = 100

for i in range(5):  # Đo 5 lần để lấy trung bình
    start_time = time.time()
    _ = model.predict(x_test[:batch_size_test], verbose=0)
    end_time = time.time()
    inference_times.append(end_time - start_time)

avg_inference_time = np.mean(inference_times)
inference_time_per_sample = avg_inference_time / batch_size_test

# Đo memory usage
process = psutil.Process()
memory_usage_mb = process.memory_info().rss / 1024 / 1024

# Model size
model_size_mb = os.path.getsize('bilstm/bilstm_attention_model.h5') / (1024 * 1024)

print("2. PERFORMANCE METRICS:")
print(f"   - Estimated Training Time: {estimated_training_time:.2f} seconds")
print(f"   - Average Inference Time (100 samples): {avg_inference_time:.4f} seconds")
print(f"   - Inference Time per Sample: {inference_time_per_sample*1000:.2f} ms")
print(f"   - Memory Usage: {memory_usage_mb:.2f} MB")
print(f"   - Model Size: {model_size_mb:.2f} MB")

performance_metrics = {
    'estimated_training_time_seconds': float(estimated_training_time),
    'avg_inference_time_100_samples': float(avg_inference_time),
    'inference_time_per_sample_ms': float(inference_time_per_sample * 1000),
    'memory_usage_mb': float(memory_usage_mb),
    'model_size_mb': float(model_size_mb),
    'epochs_trained': int(epochs_trained)
}

#### 12.3 Khả năng tổng quát hóa (Generalization)

In [ ]:
# 3. GENERALIZATION METRICS

# Train vs Validation performance
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

# Overfitting indicators
acc_gap = final_train_acc - final_val_acc
loss_gap = final_val_loss - final_train_loss

# Training stability (variance in last few epochs)
last_5_val_acc = history.history['val_accuracy'][-5:]
val_acc_variance = np.var(last_5_val_acc) if len(last_5_val_acc) >= 5 else 0

# Best validation accuracy
best_val_acc = max(history.history['val_accuracy'])
best_val_acc_epoch = history.history['val_accuracy'].index(best_val_acc) + 1

print("3. GENERALIZATION METRICS:")
print(f"   - Final Train Accuracy: {final_train_acc:.4f}")
print(f"   - Final Validation Accuracy: {final_val_acc:.4f}")
print(f"   - Accuracy Gap (Train-Val): {acc_gap:.4f}")
print(f"   - Loss Gap (Val-Train): {loss_gap:.4f}")
print(f"   - Best Validation Accuracy: {best_val_acc:.4f} (Epoch {best_val_acc_epoch})")
print(f"   - Validation Accuracy Variance (last 5): {val_acc_variance:.6f}")

# Overfitting assessment
if acc_gap > 0.05:
    overfitting_status = "Likely Overfitting"
elif acc_gap < -0.02:
    overfitting_status = "Possible Underfitting"
else:
    overfitting_status = "Good Generalization"

print(f"   - Overfitting Assessment: {overfitting_status}")

generalization_metrics = {
    'final_train_accuracy': float(final_train_acc),
    'final_validation_accuracy': float(final_val_acc),
    'final_train_loss': float(final_train_loss),
    'final_validation_loss': float(final_val_loss),
    'accuracy_gap': float(acc_gap),
    'loss_gap': float(loss_gap),
    'best_validation_accuracy': float(best_val_acc),
    'best_validation_accuracy_epoch': int(best_val_acc_epoch),
    'validation_accuracy_variance': float(val_acc_variance),
    'overfitting_status': overfitting_status
}

#### 12.4 Độ phức tạp mô hình (Model Complexity)

In [ ]:
# 4. MODEL COMPLEXITY METRICS

# Số parameters
total_params = model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable_params = total_params - trainable_params

# Số layers
total_layers = len(model.layers)

# FLOPS estimation (rough)
def estimate_flops():
    flops = 0
    # Embedding layer
    flops += MAX_FEATURES * EMBEDDING_DIM
    # LSTM layers (approximation)
    flops += 4 * LSTM_UNITS * EMBEDDING_DIM * MAXLEN * 2  # BiLSTM
    flops += 4 * LSTM_UNITS * LSTM_UNITS * MAXLEN * 2     # Second BiLSTM
    # Dense layers
    flops += (LSTM_UNITS * 2) * 64 + 64 * 32 + 32 * 1
    return flops

estimated_flops = estimate_flops()

print("4. MODEL COMPLEXITY METRICS:")
print(f"   - Total Parameters: {total_params:,}")
print(f"   - Trainable Parameters: {trainable_params:,}")
print(f"   - Non-trainable Parameters: {non_trainable_params:,}")
print(f"   - Total Layers: {total_layers}")
print(f"   - Estimated FLOPs: {estimated_flops:,}")
print(f"   - Model Size: {model_size_mb:.2f} MB")

# Architecture details
layer_details = []
for i, layer in enumerate(model.layers):
    layer_info = {
        'layer_index': i,
        'layer_name': layer.name,
        'layer_type': type(layer).__name__,
        'output_shape': str(layer.output_shape),
        'param_count': layer.count_params()
    }
    layer_details.append(layer_info)

complexity_metrics = {
    'total_parameters': int(total_params),
    'trainable_parameters': int(trainable_params),
    'non_trainable_parameters': int(non_trainable_params),
    'total_layers': int(total_layers),
    'estimated_flops': int(estimated_flops),
    'model_size_mb': float(model_size_mb),
    'layer_details': layer_details
}

#### 12.5 Tính ổn định và độ tin cậy (Stability & Reliability)

In [ ]:
# 5. STABILITY & RELIABILITY METRICS

# Confidence analysis
confidence_scores = np.abs(y_pred_prob.flatten() - 0.5)  # Distance from 0.5
avg_confidence = np.mean(confidence_scores)
confidence_std = np.std(confidence_scores)

# High confidence predictions (> 0.3 distance from 0.5)
high_confidence_mask = confidence_scores > 0.3
high_confidence_accuracy = accuracy_score(y_test[high_confidence_mask], 
                                         y_pred[high_confidence_mask]) if np.any(high_confidence_mask) else 0

# Low confidence predictions
low_confidence_mask = confidence_scores <= 0.1
low_confidence_count = np.sum(low_confidence_mask)
low_confidence_accuracy = accuracy_score(y_test[low_confidence_mask], 
                                        y_pred[low_confidence_mask]) if np.any(low_confidence_mask) else 0

# Error analysis
correct_predictions = (y_pred == y_test)
error_rate_by_confidence = []
confidence_bins = np.linspace(0, 0.5, 6)
for i in range(len(confidence_bins)-1):
    mask = (confidence_scores >= confidence_bins[i]) & (confidence_scores < confidence_bins[i+1])
    if np.any(mask):
        error_rate = 1 - accuracy_score(y_test[mask], y_pred[mask])
        error_rate_by_confidence.append(error_rate)
    else:
        error_rate_by_confidence.append(0)

print("5. STABILITY & RELIABILITY METRICS:")
print(f"   - Average Confidence: {avg_confidence:.4f}")
print(f"   - Confidence Std: {confidence_std:.4f}")
print(f"   - High Confidence Accuracy (>0.3): {high_confidence_accuracy:.4f}")
print(f"   - Low Confidence Count (≤0.1): {low_confidence_count}")
print(f"   - Low Confidence Accuracy: {low_confidence_accuracy:.4f}")

reliability_metrics = {
    'average_confidence': float(avg_confidence),
    'confidence_std': float(confidence_std),
    'high_confidence_accuracy': float(high_confidence_accuracy),
    'low_confidence_count': int(low_confidence_count),
    'low_confidence_accuracy': float(low_confidence_accuracy),
    'error_rate_by_confidence_bins': [float(x) for x in error_rate_by_confidence]
}

### 13. Vẽ biểu đồ thống kê tổng hợp

Tạo các biểu đồ để trực quan hóa tất cả các thông số đánh giá:

In [ ]:
# COMPREHENSIVE VISUALIZATION
plt.style.use('default')
fig = plt.figure(figsize=(20, 16))

# 1. Accuracy Metrics Bar Chart
ax1 = plt.subplot(3, 4, 1)
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
metrics_values = [accuracy, precision, recall, f1, auc_roc]
bars = plt.bar(metrics_names, metrics_values, color=['#2E86C1', '#28B463', '#F39C12', '#E74C3C', '#8E44AD'])
plt.title('Accuracy Metrics')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.ylim(0, 1)
for bar, value in zip(bars, metrics_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# 2. Confusion Matrix Heatmap
ax2 = plt.subplot(3, 4, 2)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# 3. ROC Curve
ax3 = plt.subplot(3, 4, 3)
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc_roc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

# 4. Training History
ax4 = plt.subplot(3, 4, 4)
epochs = range(1, len(history.history['accuracy']) + 1)
plt.plot(epochs, history.history['accuracy'], 'bo-', label='Training Accuracy', markersize=4)
plt.plot(epochs, history.history['val_accuracy'], 'ro-', label='Validation Accuracy', markersize=4)
plt.title('Training History')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. Performance Metrics
ax5 = plt.subplot(3, 4, 5)
perf_names = ['Inference Time\n(ms/sample)', 'Memory Usage\n(MB)', 'Model Size\n(MB)']
perf_values = [inference_time_per_sample*1000, memory_usage_mb, model_size_mb]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = plt.bar(perf_names, perf_values, color=colors)
plt.title('Performance Metrics')
plt.ylabel('Value')
for bar, value in zip(bars, perf_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(perf_values)*0.01, 
             f'{value:.2f}', ha='center', va='bottom', fontweight='bold')

# 6. Model Complexity
ax6 = plt.subplot(3, 4, 6)
complexity_names = ['Parameters\n(K)', 'Layers', 'FLOPs\n(M)']
complexity_values = [total_params/1000, total_layers, estimated_flops/1000000]
bars = plt.bar(complexity_names, complexity_values, color=['#96CEB4', '#FFEAA7', '#DDA0DD'])
plt.title('Model Complexity')
plt.ylabel('Count')
for bar, value in zip(bars, complexity_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(complexity_values)*0.01, 
             f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

# 7. Confidence Distribution
ax7 = plt.subplot(3, 4, 7)
plt.hist(y_pred_prob, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Decision Boundary')
plt.title('Prediction Confidence Distribution')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.legend()

# 8. Error Rate by Confidence
ax8 = plt.subplot(3, 4, 8)
confidence_bin_centers = [(confidence_bins[i] + confidence_bins[i+1])/2 for i in range(len(confidence_bins)-1)]
plt.plot(confidence_bin_centers, error_rate_by_confidence, 'ro-', linewidth=2, markersize=6)
plt.title('Error Rate by Confidence Level')
plt.xlabel('Confidence Level')
plt.ylabel('Error Rate')
plt.grid(True, alpha=0.3)

# 9. Train vs Validation Comparison
ax9 = plt.subplot(3, 4, 9)
comparison_metrics = ['Accuracy', 'Loss']
train_values = [final_train_acc, final_train_loss]
val_values = [final_val_acc, final_val_loss]
x = np.arange(len(comparison_metrics))
width = 0.35
plt.bar(x - width/2, train_values, width, label='Training', color='lightblue')
plt.bar(x + width/2, val_values, width, label='Validation', color='lightcoral')
plt.title('Train vs Validation')
plt.xlabel('Metrics')
plt.ylabel('Value')
plt.xticks(x, comparison_metrics)
plt.legend()

# 10. Loss History
ax10 = plt.subplot(3, 4, 10)
plt.plot(epochs, history.history['loss'], 'bo-', label='Training Loss', markersize=4)
plt.plot(epochs, history.history['val_loss'], 'ro-', label='Validation Loss', markersize=4)
plt.title('Loss History')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# 11. Classification Performance by Class
ax11 = plt.subplot(3, 4, 11)
class_metrics = ['Precision', 'Recall', 'F1-Score']
negative_scores = [tn/(tn+fp), tn/(tn+fn), 2*(tn/(tn+fp))*(tn/(tn+fn))/((tn/(tn+fp))+(tn/(tn+fn)))]
positive_scores = [precision, recall, f1]
x = np.arange(len(class_metrics))
plt.bar(x - width/2, negative_scores, width, label='Negative Class', color='lightcoral')
plt.bar(x + width/2, positive_scores, width, label='Positive Class', color='lightgreen')
plt.title('Performance by Class')
plt.xlabel('Metrics')
plt.ylabel('Score')
plt.xticks(x, class_metrics)
plt.legend()
plt.ylim(0, 1)

# 12. Model Size Breakdown
ax12 = plt.subplot(3, 4, 12)
param_breakdown = ['Trainable', 'Non-trainable']
param_values = [trainable_params/1000, non_trainable_params/1000]
colors = ['#FF9999', '#66B2FF']
wedges, texts, autotexts = plt.pie(param_values, labels=param_breakdown, colors=colors, 
                                  autopct='%1.1f%%', startangle=90)
plt.title('Parameters Breakdown (K)')

plt.tight_layout(pad=3.0)
plt.suptitle('BiLSTM + Attention Model - Comprehensive Analysis', fontsize=16, y=0.98)
plt.savefig('bilstm/comprehensive_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Biểu đồ tổng hợp đã được lưu vào 'bilstm/comprehensive_analysis.png'")

### 14. Lưu tất cả các thông số để so sánh với BERT

Tổng hợp và lưu tất cả các metrics vào file JSON để so sánh với mô hình BERT:

In [ ]:
# TỔNG HỢP TẤT CẢ METRICS ĐỂ SO SÁNH
comprehensive_metrics = {
    'model_info': {
        'model_name': 'BiLSTM + Attention',
        'dataset': 'IMDb Movie Reviews',
        'task': 'Binary Sentiment Classification',
        'framework': 'TensorFlow/Keras',
        'training_date': datetime.now().isoformat(),
        'hyperparameters': {
            'max_features': MAX_FEATURES,
            'max_length': MAXLEN,
            'embedding_dim': EMBEDDING_DIM,
            'lstm_units': LSTM_UNITS,
            'batch_size': BATCH_SIZE,
            'epochs': EPOCHS
        }
    },
    
    # 1. Accuracy Metrics
    'accuracy_metrics': accuracy_metrics,
    
    # 2. Performance Metrics  
    'performance_metrics': performance_metrics,
    
    # 3. Generalization Metrics
    'generalization_metrics': generalization_metrics,
    
    # 4. Model Complexity
    'complexity_metrics': complexity_metrics,
    
    # 5. Reliability Metrics
    'reliability_metrics': reliability_metrics,
    
    # 6. Training History
    'training_history': {
        'train_accuracy_history': [float(x) for x in history.history['accuracy']],
        'val_accuracy_history': [float(x) for x in history.history['val_accuracy']],
        'train_loss_history': [float(x) for x in history.history['loss']],
        'val_loss_history': [float(x) for x in history.history['val_loss']]
    },
    
    # 7. Additional Analysis
    'additional_metrics': {
        'dataset_size': {
            'train_samples': len(x_train),
            'test_samples': len(x_test),
            'total_samples': len(x_train) + len(x_test)
        },
        'class_distribution': {
            'train_positive': int(np.sum(y_train)),
            'train_negative': int(len(y_train) - np.sum(y_train)),
            'test_positive': int(np.sum(y_test)),
            'test_negative': int(len(y_test) - np.sum(y_test))
        },
        'prediction_stats': {
            'predicted_positive': int(np.sum(y_pred)),
            'predicted_negative': int(len(y_pred) - np.sum(y_pred)),
            'avg_positive_confidence': float(np.mean(y_pred_prob[y_test == 1])),
            'avg_negative_confidence': float(np.mean(1 - y_pred_prob[y_test == 0]))
        }
    }
}

# Lưu vào file JSON
with open('bilstm/bilstm_comprehensive_metrics.json', 'w') as f:
    json.dump(comprehensive_metrics, f, indent=2, ensure_ascii=False)

print("✅ Tất cả metrics đã được lưu vào 'bilstm/bilstm_comprehensive_metrics.json'")

# Tạo summary report
summary_report = f'''
=================================================================
            BILSTM + ATTENTION MODEL - SUMMARY REPORT
=================================================================

📊 MODEL PERFORMANCE:
   • Test Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)
   • Precision: {precision:.4f}
   • Recall: {recall:.4f}
   • F1-Score: {f1:.4f}
   • AUC-ROC: {auc_roc:.4f}

⚡ PERFORMANCE:
   • Training Time: ~{estimated_training_time:.0f} seconds
   • Inference Time: {inference_time_per_sample*1000:.2f} ms/sample
   • Model Size: {model_size_mb:.2f} MB
   • Memory Usage: {memory_usage_mb:.2f} MB

🔧 MODEL COMPLEXITY:
   • Total Parameters: {total_params:,}
   • Trainable Parameters: {trainable_params:,}
   • Total Layers: {total_layers}
   • Estimated FLOPs: {estimated_flops:,}

📈 GENERALIZATION:
   • Best Validation Accuracy: {best_val_acc:.4f}
   • Overfitting Status: {overfitting_status}
   • Train-Val Accuracy Gap: {acc_gap:.4f}

🎯 RELIABILITY:
   • Average Confidence: {avg_confidence:.4f}
   • High Confidence Accuracy: {high_confidence_accuracy:.4f}
   • Low Confidence Predictions: {low_confidence_count}

=================================================================
Model và tất cả metrics đã được lưu trong thư mục 'bilstm/'
Sẵn sàng để so sánh với BERT!
=================================================================
'''
print(summary_report)
# Lưu summary report
with open('bilstm/summary_report.txt', 'w', encoding='utf-8') as f:
    f.write(summary_report)
print("✅ Summary report đã được lưu vào 'bilstm/summary_report.txt'")


### 15. Tạo script so sánh với BERT

Tạo sẵn script để so sánh với mô hình BERT khi có dữ liệu:

In [ ]:
# TẠO SCRIPT SO SÁNH VỚI BERT
comparison_script = '''
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def compare_models(bilstm_metrics_path, bert_metrics_path):
    """
    So sánh BiLSTM với BERT từ các file metrics JSON
    """
    # Load metrics
    with open(bilstm_metrics_path, 'r') as f:
        bilstm_metrics = json.load(f)
    
    with open(bert_metrics_path, 'r') as f:
        bert_metrics = json.load(f)
    
    # Tạo bảng so sánh
    comparison_data = {
        'Metric': [
            'Test Accuracy',
            'Precision', 
            'Recall',
            'F1-Score',
            'AUC-ROC',
            'Model Size (MB)',
            'Parameters',
            'Inference Time (ms)',
            'Training Time (s)',
            'Memory Usage (MB)'
        ],
        'BiLSTM + Attention': [
            bilstm_metrics['accuracy_metrics']['accuracy'],
            bilstm_metrics['accuracy_metrics']['precision'],
            bilstm_metrics['accuracy_metrics']['recall'],
            bilstm_metrics['accuracy_metrics']['f1_score'],
            bilstm_metrics['accuracy_metrics']['auc_roc'],
            bilstm_metrics['performance_metrics']['model_size_mb'],
            bilstm_metrics['complexity_metrics']['total_parameters'],
            bilstm_metrics['performance_metrics']['inference_time_per_sample_ms'],
            bilstm_metrics['performance_metrics']['estimated_training_time_seconds'],
            bilstm_metrics['performance_metrics']['memory_usage_mb']
        ],
        'BERT': [
            # Sẽ được điền khi có dữ liệu BERT
            bert_metrics.get('accuracy_metrics', {}).get('accuracy', 0),
            bert_metrics.get('accuracy_metrics', {}).get('precision', 0),
            bert_metrics.get('accuracy_metrics', {}).get('recall', 0),
            bert_metrics.get('accuracy_metrics', {}).get('f1_score', 0),
            bert_metrics.get('accuracy_metrics', {}).get('auc_roc', 0),
            bert_metrics.get('performance_metrics', {}).get('model_size_mb', 0),
            bert_metrics.get('complexity_metrics', {}).get('total_parameters', 0),
            bert_metrics.get('performance_metrics', {}).get('inference_time_per_sample_ms', 0),
            bert_metrics.get('performance_metrics', {}).get('training_time_seconds', 0),
            bert_metrics.get('performance_metrics', {}).get('memory_usage_mb', 0)
        ]
    }
    
    df = pd.DataFrame(comparison_data)
    print("MODEL COMPARISON TABLE:")
    print("=" * 60)
    print(df.to_string(index=False))
    
    # Vẽ biểu đồ so sánh
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy metrics
    accuracy_metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
    bilstm_acc = df.iloc[0:5]['BiLSTM + Attention'].values
    bert_acc = df.iloc[0:5]['BERT'].values
    
    x = np.arange(len(accuracy_metrics))
    width = 0.35
    
    axes[0,0].bar(x - width/2, bilstm_acc, width, label='BiLSTM', color='lightblue')
    axes[0,0].bar(x + width/2, bert_acc, width, label='BERT', color='lightcoral')
    axes[0,0].set_title('Accuracy Metrics Comparison')
    axes[0,0].set_xlabel('Metrics')
    axes[0,0].set_ylabel('Score')
    axes[0,0].set_xticks(x)
    axes[0,0].set_xticklabels(accuracy_metrics, rotation=45)
    axes[0,0].legend()
    axes[0,0].set_ylim(0, 1)
    
    # Performance metrics
    perf_metrics = ['Model Size (MB)', 'Inference Time (ms)', 'Memory Usage (MB)']
    bilstm_perf = [df.iloc[5]['BiLSTM + Attention'], df.iloc[7]['BiLSTM + Attention'], df.iloc[9]['BiLSTM + Attention']]
    bert_perf = [df.iloc[5]['BERT'], df.iloc[7]['BERT'], df.iloc[9]['BERT']]
    
    x = np.arange(len(perf_metrics))
    axes[0,1].bar(x - width/2, bilstm_perf, width, label='BiLSTM', color='lightgreen')
    axes[0,1].bar(x + width/2, bert_perf, width, label='BERT', color='orange')
    axes[0,1].set_title('Performance Metrics Comparison')
    axes[0,1].set_xlabel('Metrics')  
    axes[0,1].set_ylabel('Value')
    axes[0,1].set_xticks(x)
    axes[0,1].set_xticklabels(perf_metrics, rotation=45)
    axes[0,1].legend()
    
    # Parameters comparison
    models = ['BiLSTM + Attention', 'BERT']
    params = [df.iloc[6]['BiLSTM + Attention'], df.iloc[6]['BERT']]
    
    axes[1,0].bar(models, params, color=['purple', 'gold'])
    axes[1,0].set_title('Model Parameters Comparison')
    axes[1,0].set_ylabel('Number of Parameters')
    
    # Training time comparison
    train_times = [df.iloc[8]['BiLSTM + Attention'], df.iloc[8]['BERT']]
    
    axes[1,1].bar(models, train_times, color=['teal', 'crimson'])
    axes[1,1].set_title('Training Time Comparison')
    axes[1,1].set_ylabel('Training Time (seconds)')
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return df

# Sử dụng:
# df = compare_models('bilstm/bilstm_comprehensive_metrics.json', 'bert/bert_comprehensive_metrics.json')
'''

# Lưu script
with open('bilstm/compare_models.py', 'w', encoding='utf-8') as f:
    f.write(comparison_script)

print("✅ Script so sánh đã được tạo: 'bilstm/compare_models.py'")
print("\n📋 Files đã được tạo trong thư mục 'bilstm/':")
print("   • bilstm_attention_model.h5 - Mô hình đã train")
print("   • word_index.pkl - Tokenizer info")
print("   • bilstm_comprehensive_metrics.json - Tất cả metrics")
print("   • summary_report.txt - Báo cáo tóm tắt")
print("   • comprehensive_analysis.png - Biểu đồ tổng hợp")
print("   • compare_models.py - Script so sánh với BERT")

print("\n🔄 Để so sánh với BERT sau này, chạy:")
print("   from bilstm.compare_models import compare_models")
print("   compare_models('bilstm/bilstm_comprehensive_metrics.json', 'bert_metrics.json')")

# Hiển thị thông tin thư mục
import os
print(f"\n📁 Nội dung thư mục 'bilstm':")
for file in os.listdir('bilstm'):
    file_path = os.path.join('bilstm', file)
    size = os.path.getsize(file_path) / 1024  # KB
    print(f"   • {file} ({size:.1f} KB)")